In [ ]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import numpy as np
from keras.models import load_model
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
# from dataset import prepare_gtsf_data, compute_akpd_score
# from augment import augment
from weight_estimation.weight_estimator import WeightEstimator
from weight_estimation.utils import CameraMetadata

from matplotlib import pyplot as plt


In [ ]:
s3 = S3AccessUtils('/root/data')

In [ ]:
rds = RDSAccessUtils()
query = """
    select * from prod.biomass_computations
    where pen_id=145
    and captured_at between '2021-02-02' and '2021-02-08'
    and akpd_score > 0.9;
"""
df = rds.extract_from_database(query)

In [ ]:
df.camera_metadata.iloc[0]

In [ ]:
weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

pred_weights = []
count = 0

for idx, row in df.iterrows():
    ann = row.annotation
    camera_metadata = row.camera_metadata
    cm = CameraMetadata(
        focal_length=camera_metadata['focalLength'],
        focal_length_pixel=camera_metadata['focalLengthPixel'],
        baseline_m=camera_metadata['baseline'],
        pixel_count_width=camera_metadata['pixelCountWidth'],
        pixel_count_height=camera_metadata['pixelCountHeight'],
        image_sensor_width=camera_metadata['imageSensorWidth'],
        image_sensor_height=camera_metadata['imageSensorHeight']
    )
    
    weight, _, _ = weight_estimator.predict(ann, cm)
    pred_weights.append(weight)
    
    if count % 1000 == 0:
        print(count)
    count += 1
    

In [ ]:
df['weight'] = pred_weights

In [ ]:
(df.weight - df.estimated_weight_g).abs().sum()